
<img src='https://www.bytelion.com/wp-content/uploads/2015/12/python-banner.png'>

# Modern Python Development


Content,

- Pyenv
- Poetry 
- Pytest
- Coverage
- Nox 
- Pylint
- flake8 
- xdcote
- readthedocs



In [2]:
#start from scratch 
!cd /Users/rjbrooker/Documents/github_repos/modern-python-tutorial/
!rm -r -f insult-ai

# 1. Enviroment & Dependencies

## 1.1. [pyenv](https://github.com/pyenv/pyenv)  

Python version manager. 
- Easily switch between multiple versions of Python. 
- Supports per-project Python versions.
- Single-purpose tool that does one thing well.

Install : 

In [ ]:
# install pyenv
!brew install pyenv
!brew install pyenv-virtualenv

# set enviroment variables (see https://github.com/pyenv/pyenv.)
!echo 'eval "$(pyenv init --path)"' >> ~/.zprofile

# check version
!pyenv --version

In [ ]:
!open ~/.zprofile

Open '\~/.zshrc' to see the changes, 

#### 1.1.1 Installing Multiple Python Versions

In [ ]:
# there is currntly an issue with big sure so a patch is needed
!pyenv install 3.9.5
!pyenv install 3.7.7 --patch < <(curl -sSL https://github.com/python/cpython/commit/8ea6353.patch)

In [ ]:
!pyenv versions

In [ ]:
#!pyenv global 3.9.5

For installing anaconda and pyenv see, 
https://stackoverflow.com/questions/38217545/what-is-the-difference-between-pyenv-virtualenv-anaconda

<br><Br><br><Br>

<img src="https://davidr.in/static/media/poetry.f8fc3dd5.png" style="height:200px ; float: left;"> 
<br><br><br><br>

## 1.2 [Poetry](https://python-poetry.org/)

Python Dependency manager. It is similar to npm and yarn. Improvment over setuptools. 



In [ ]:
#install 
!brew install poetry -q
!poetry --version 

### 1.2.1. Poetry - Creating a Project

Initialize your project 

In [3]:
!poetry new insult-ai 

Created package insult_ai in insult-ai


In [4]:
#!brew install tree  > /dev/null
!tree  

.
├── Modern\ Python\ Development.ipynb
└── insult-ai
    ├── README.rst
    ├── insult_ai
    │   └── __init__.py
    ├── pyproject.toml
    └── tests
        ├── __init__.py
        └── test_insult_ai.py

3 directories, 6 files


In [5]:
!open $(pwd)

In [9]:
%cd insult-ai

/Users/rjbrooker/Documents/github_repos/modern-python-tutorial/insult-ai


### 1.2.1. Poetry - Creating a Project

Init repo and setup origin, 

In [ ]:
!git init
!git remote set-url origin git@github.com:rjbrooker-anghami/insult_ai.git

In [ ]:
!git remote add origin git@github.com:rjbrooker-anghami/insult_ai.git

Commit code,

In [ ]:
!git add . 
!git commit -m "init"
!git push origin master

### 1.2.1 Poetry - Running Virtual Environments

Poetry lets you created a virtual environment dedicated to your project

Install enviroment, 

In [ ]:
!poetry install -q 

In [ ]:
# run python/jupyter inside your enviroment
!poetry run jupyter notebook

In [ ]:
# run the enviroment shell 
!poetry shell 

In [ ]:
!pip install . 

In [ ]:
import insult_ai
insult_ai.__version__

### 2.2 Poetry - Installing Dependencies

In [10]:
!poetry add click

Using version ^8.0.1 for click

Updating dependencies
Resolving dependencies... (5.7s)Resolving dependencies... (3.0s)Resolving dependencies... (4.3s)Resolving dependencies... (4.4s)

Writing lock file

No dependencies to install or update


This will, 
- Downloaded and installed the package into the virtual environment.
- Adds the exact version to the lock file poetry.lock.
- Adds a less constrained version to pyproject.toml.

In [7]:
!open -a TextEdit pyproject.toml

The file /Users/rjbrooker/Documents/github_repos/modern-python-tutorial/pyproject.toml does not exist.


# 2. Making a Package

<br><br><br><br><br><br>
<img src="https://click.palletsprojects.com/en/8.0.x/_images/click-logo.png" style="height:120px ; float: left;"> 
<br>

## 2.1 [Click.py](https://click.palletsprojects.com/en/8.0.x/)

Creating a command line tool.  

In [11]:
%%writefile insult_ai/console.py

import click
from . import __version__

@click.command()
@click.option('--message', default="", help='A message to send to insult-ai.')
@click.version_option(version=__version__)
def insult_me(message : str ):
    """Insult-ai Python project."""
    click.echo(f"'{message}'?")


Writing insult_ai/console.py


In [12]:
%%writefile -a pyproject.toml

[tool.poetry.scripts]
insult-ai = "insult_ai.console:insult_me"

Appending to pyproject.toml


Install the package

In [13]:
!pip install . -q

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
You should consider upgrading via the '/Users/rjbrooker/.pyenv/versions/3.9.5/bin/python3.9 -m pip install --upgrade pip' command.


In [14]:
!insult-ai --message "Hello"

'Hello'?


In [15]:
!poetry install -q
!poetry run insult-ai --message "Hello"

'Hello'?


In [16]:
!poetry run insult-ai --help

Usage: insult-ai [OPTIONS]

  Insult-ai Python project.

Options:
  --message TEXT  A message to send to insult-ai.
  --version       Show the version and exit.
  --help          Show this message and exit.


## 2.2 [🙊 Detoxify](https://pypi.org/project/detoxify/)

Adding ml to our tool.

- [Detoxify](https://pypi.org/project/detoxify/) - pretrained sentiment analysis model.

In [17]:
!poetry add pandas detoxify -q

Add detoxify to our function, 

In [97]:
%%writefile insult_ai/console.py
import click
import json 
from . import __version__
from detoxify import Detoxify
import pandas as pd 

@click.command()
@click.option('--message', default="", help='A message to send to insult-ai.')
@click.version_option(version=__version__)
def insult_me(message : str ):
    """Insult-ai Python project."""
    
    #load model
    model = Detoxify('original')
    
    #predict toxicity
    results = model.predict(message)
    
    #echo results
    click.echo(pd.Series(results).sort_values(ascending=False))


Overwriting insult_ai/console.py


In [98]:
!poetry install -q

In [99]:
!poetry run insult-ai --message "I really don't hate your hair"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 986, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 680, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 855, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/rjbrooker/Documents/github_repos/modern-python-tutorial/insult-ai/insult_ai/console.py", line 3, in <module>
    from . import __version__
ImportError: cannot import name '__version__' from 'insult_ai' (/Users/rjbrooker/Documents/github_repos/modern-python-tutorial/insult-ai/insul

## 2.3. Poetry -  [PyPi](https://pypi.org/) Publishing 

Build the package 

In [29]:
!poetry build 

Building insult-ai (0.2.0)
  - Building sdist
  - Built insult-ai-0.2.0.tar.gz
  - Building wheel
  - Built insult_ai-0.2.0-py3-none-any.whl


In [30]:
from getpass import getpass
password = getpass()
!poetry publish -u rjbrooker -p $password

········

Publishing insult-ai (0.2.0) to PyPI
 - Uploading insult-ai-0.2.0.tar.gz 100%
 - Uploading insult_ai-0.2.0-py3-none-any.whl 100%


# 4. Unit Tests 

Unit tests, as the name says, verify the functionality of a unit of code, 


<br><br><br><br><br><br>
<img src="https://docs.pytest.org/en/6.2.x/_static/pytest1.png" style="height:160px ; float: left;"> 
<br><br><br><br>

## 4.1 [Pytest](https://docs.pytest.org/en/6.2.x/)

In [31]:
!poetry add --dev pytest -q

In [32]:
%%writefile tests/test_insult_ai.py

import click.testing
import pytest
from insult_ai import console

@pytest.fixture
def runner():
    """Reusable helper function."""
    return click.testing.CliRunner()

def test_insult_me_succeeds(runner):
    result = runner.invoke(console.insult_me, ['--message="You look nice."'])
    assert result.exit_code == 0


Overwriting tests/test_insult_ai.py


In [33]:
!poetry run pytest

============================= test session starts ==============================
platform darwin -- Python 3.9.5, pytest-5.4.3, py-1.10.0, pluggy-0.13.1
rootdir: /Users/rjbrooker/Documents/github_repos/modern-python-tutorial/insult-ai
plugins: cov-2.12.1, mock-3.6.1, xdoctest-0.15.4
collected 1 item                                                               

tests/test_insult_ai.py .                                                [100%]

============================== 1 passed in 7.79s ===============================


<br><br><br><br><br><br><br><br>
<img src="https://coverage.readthedocs.io/en/coverage-5.5/_static/sleepy-snake-circle-150.png" style="height:160px ; float: left;"> 
<br><br><br><br>

## 4.2 [Coverage.py](https://coverage.readthedocs.io/en/coverage-5.5/)
Allows you to see what percentage of your code is covered by tests. 


In [34]:
!poetry add coverage pytest-cov -q

Create configurations file,

In [35]:
%%writefile -a pyproject.toml

[tool.coverage.paths]
source = ["insult_ai", "*/site-packages"]

[tool.coverage.run]
branch = true
source = ["insult_ai"]

[tool.coverage.report]
show_missing = true
fail_under = 99

Appending to pyproject.toml


Calculate coverage, 

In [36]:
!poetry run pytest --cov

============================= test session starts ==============================
platform darwin -- Python 3.9.5, pytest-5.4.3, py-1.10.0, pluggy-0.13.1
rootdir: /Users/rjbrooker/Documents/github_repos/modern-python-tutorial/insult-ai
plugins: cov-2.12.1, mock-3.6.1, xdoctest-0.15.4
collected 1 item                                                               

tests/test_insult_ai.py .                                                [100%]

---------- coverage: platform darwin, python 3.9.5-final-0 -----------
Name                    Stmts   Miss Branch BrPart  Cover   Missing
-------------------------------------------------------------------
insult_ai/__init__.py       1      0      0      0   100%
insult_ai/console.py       12      0      0      0   100%
-------------------------------------------------------------------
TOTAL                      13      0      0      0   100%

Required test coverage of 99.0% reached. Total coverage: 100.00%

============================== 1 passe

Aiming for 100% code coverage is good practice. However his number does not imply that your test suite has meaningful test cases for all uses and misuses of your program.



### 4.2.1 Coverage Reporting
Tracking test coverage, 

- [codecov](https://github.com/marketplace/codecov)
- [Coveralls](https://coveralls.io/)

<br><br><br><br><br><br><br><br>
<img src="https://nox.thea.codes/en/stable/_static/alice.png" style="height:160px ; float: left;"> 
<br><br><br><br>

## 4.4 [Nox.py](https://nox.thea.codes/en/stable/)


Automates testing in multiple Python environments. Nox makes it easy to run any kind of job in an isolated environment,


In [ ]:
# install 
!brew install nox -q

Create nox file, 

In [37]:
%%writefile noxfile.py

import nox

@nox.session(python=["3.8", "3.9"])
def tests(session):
    session.run("poetry", "install", external=True)
    session.run("pytest", "--cov")


Writing noxfile.py


In [38]:
!nox

nox > Running session tests-3.8
nox > Creating virtual environment (virtualenv) using python3.8 in .nox/tests-3-8
nox > poetry install
Installing dependencies from lock file

Package operations: 35 installs, 0 updates, 0 removals

  • Installing certifi (2021.5.30): Pending...
  • Installing chardet (4.0.0): Pending...
  • Installing idna (2.10): Pending...
  • Installing pyparsing (2.4.7): Pending...
  • Installing urllib3 (1.26.5): Pending...
  • Installing chardet (4.0.0): Pending...
  • Installing idna (2.10): Pending...
  • Installing pyparsing (2.4.7): Pending...
  • Installing urllib3 (1.26.5): Pending...
  • Installing certifi (2021.5.30): Installing...
  • Installing chardet (4.0.0): Pending...
  • Installing idna (2.10): Pending...
  • Installing pyparsing (2.4.7): Pending...
  • Installing urllib3 (1.26.5): Pending...
  • Installing urllib3 (1.26.5): Pending...
  • Installing pyparsing (2.4.7): Installing...
  • Installing urllib3 (1.26.5): Pending...
  • Installing pyparsin

  • Installing click (8.0.1): Installing...
  • Installing filelock (3.0.12): Pending...
  • Installing joblib (1.0.1): Pending...
  • Installing packaging (20.9): Pending...
  • Installing regex (2021.4.4): Pending...
  • Installing requests (2.25.1): Pending...
  • Installing six (1.16.0): Pending...
  • Installing tqdm (4.61.1): Pending...
  • Installing typing-extensions (3.10.0.0): Pending...
  • Installing joblib (1.0.1): Pending...
  • Installing packaging (20.9): Pending...
  • Installing regex (2021.4.4): Pending...
  • Installing requests (2.25.1): Pending...
  • Installing six (1.16.0): Pending...
  • Installing tqdm (4.61.1): Pending...
  • Installing typing-extensions (3.10.0.0): Pending...
  • Installing filelock (3.0.12): Installing...
  • Installing joblib (1.0.1): Pending...
  • Installing packaging (20.9): Pending...
  • Installing regex (2021.4.4): Pending...
  • Installing requests (2.25.1): Pending...
  • Installing six (1.16.0): Pending...
  • Installing tqdm (4.6

  • Installing joblib (1.0.1): Installing...
  • Installing packaging (20.9): Installing...
  • Installing regex (2021.4.4): Installing...
  • Installing requests (2.25.1): Installing...
  • Installing six (1.16.0): Installing...
  • Installing tqdm (4.61.1): Installing...
  • Installing typing-extensions (3.10.0.0): Installing...
  • Installing filelock (3.0.12)
  • Installing joblib (1.0.1): Installing...
  • Installing packaging (20.9): Installing...
  • Installing regex (2021.4.4): Installing...
  • Installing requests (2.25.1): Installing...
  • Installing six (1.16.0): Installing...
  • Installing tqdm (4.61.1): Installing...
  • Installing typing-extensions (3.10.0.0): Installing...
  • Installing filelock (3.0.12)
  • Installing joblib (1.0.1): Installing...
  • Installing packaging (20.9): Installing...
  • Installing regex (2021.4.4): Installing...
  • Installing requests (2.25.1): Installing...
  • Installing six (1.16.0): Installing...
  • Installing tqdm (4.61.1): Installi

  • Installing huggingface-hub (0.0.12): Pending...
  • Installing more-itertools (8.8.0): Pending...
  • Installing numpy (1.21.0): Pending...
  • Installing pluggy (0.13.1): Pending...
  • Installing py (1.10.0): Pending...
  • Installing pyyaml (5.4.1): Pending...
  • Installing sacremoses (0.0.45): Pending...
  • Installing tokenizers (0.10.3): Pending...
  • Installing wcwidth (0.2.5): Pending...
  • Installing huggingface-hub (0.0.12): Pending...
  • Installing more-itertools (8.8.0): Pending...
  • Installing numpy (1.21.0): Pending...
  • Installing pluggy (0.13.1): Pending...
  • Installing py (1.10.0): Pending...
  • Installing pyyaml (5.4.1): Pending...
  • Installing sacremoses (0.0.45): Pending...
  • Installing tokenizers (0.10.3): Pending...
  • Installing wcwidth (0.2.5): Pending...
  • Installing attrs (21.2.0): Installing...
  • Installing huggingface-hub (0.0.12): Pending...
  • Installing more-itertools (8.8.0): Pending...
  • Installing numpy (1.21.0): Pending...
 

  • Installing wcwidth (0.2.5): Installing...
  • Installing tokenizers (0.10.3): Installing...
  • Installing wcwidth (0.2.5): Installing...
  • Installing py (1.10.0): Installing...
  • Installing pyyaml (5.4.1): Installing...
  • Installing sacremoses (0.0.45): Installing...
  • Installing tokenizers (0.10.3): Installing...
  • Installing wcwidth (0.2.5): Installing...
  • Installing pluggy (0.13.1)
  • Installing py (1.10.0): Installing...
  • Installing pyyaml (5.4.1): Installing...
  • Installing sacremoses (0.0.45): Installing...
  • Installing tokenizers (0.10.3): Installing...
  • Installing wcwidth (0.2.5): Installing...
  • Installing numpy (1.21.0): Pending...
  • Installing pluggy (0.13.1)
  • Installing py (1.10.0): Installing...
  • Installing pyyaml (5.4.1): Installing...
  • Installing sacremoses (0.0.45): Installing...
  • Installing tokenizers (0.10.3): Installing...
  • Installing wcwidth (0.2.5): Installing...
  • Installing more-itertools (8.8.0)
  • Installing nu

...

  • Installing tokenizers (0.10.3): Pending...
  • Installing wcwidth (0.2.5): Pending...
  • Installing huggingface-hub (0.0.12): Pending...
  • Installing more-itertools (8.8.0): Pending...
  • Installing numpy (1.21.0): Pending...
  • Installing pluggy (0.13.1): Pending...
  • Installing py (1.10.0): Pending...
  • Installing pyyaml (5.4.1): Pending...
  • Installing sacremoses (0.0.45): Pending...
  • Installing tokenizers (0.10.3): Pending...
  • Installing wcwidth (0.2.5): Pending...
  • Installing attrs (21.2.0): Installing...
  • Installing huggingface-hub (0.0.12): Pending...
  • Installing more-itertools (8.8.0): Pending...
  • Installing numpy (1.21.0): Pending...
  • Installing pluggy (0.13.1): Pending...
  • Installing py (1.10.0): Pending...
  • Installing pyyaml (5.4.1): Pending...
  • Installing sacremoses (0.0.45): Pending...
  • Installing tokenizers (0.10.3): Pending...
  • Installing wcwidth (0.2.5): Pending...
  • Installing numpy (1.21.0): Pending...
  • Installin

  • Installing sacremoses (0.0.45): Installing...
  • Installing tokenizers (0.10.3): Pending...
  • Installing wcwidth (0.2.5): Installing...
  • Installing pyyaml (5.4.1): Installing...
  • Installing sacremoses (0.0.45): Installing...
  • Installing tokenizers (0.10.3): Pending...
  • Installing wcwidth (0.2.5): Installing...
  • Installing wcwidth (0.2.5): Installing...
  • Installing tokenizers (0.10.3): Installing...
  • Installing wcwidth (0.2.5): Installing...
  • Installing pluggy (0.13.1): Installing...
  • Installing py (1.10.0): Installing...
  • Installing pyyaml (5.4.1): Installing...
  • Installing sacremoses (0.0.45): Installing...
  • Installing tokenizers (0.10.3): Installing...
  • Installing wcwidth (0.2.5): Installing...
  • Installing numpy (1.21.0): Installing...
  • Installing pluggy (0.13.1): Installing...
  • Installing py (1.10.0): Installing...
  • Installing pyyaml (5.4.1): Installing...
  • Installing sacremoses (0.0.45): Installing...
  • Installing token

  • Installing tokenizers (0.10.3): Installing...
  • Installing wcwidth (0.2.5): Installing...
  • Installing sacremoses (0.0.45)
  • Installing tokenizers (0.10.3): Installing...
  • Installing wcwidth (0.2.5): Installing...
  • Installing wcwidth (0.2.5)
  • Installing pyyaml (5.4.1): Installing...
  • Installing sacremoses (0.0.45)
  • Installing tokenizers (0.10.3): Installing...
  • Installing wcwidth (0.2.5)
  • Installing py (1.10.0)
  • Installing pyyaml (5.4.1): Installing...
  • Installing sacremoses (0.0.45)
  • Installing tokenizers (0.10.3): Installing...
  • Installing wcwidth (0.2.5)
  • Installing sacremoses (0.0.45)
  • Installing tokenizers (0.10.3): Installing...
  • Installing wcwidth (0.2.5)
  • Installing pyyaml (5.4.1)
  • Installing sacremoses (0.0.45)
  • Installing tokenizers (0.10.3): Installing...
  • Installing wcwidth (0.2.5)
  • Installing wcwidth (0.2.5)
  • Installing tokenizers (0.10.3)
  • Installing wcwidth (0.2.5)
  • Installing pluggy (0.13.1)
  •

  • Installing sentencepiece (0.1.96): Installing...
  • Installing toml (0.10.2): Installing...
  • Installing torch (1.9.0): Installing...
  • Installing transformers (4.8.0): Installing...
  • Installing pytz (2021.1): Installing...
  • Installing sentencepiece (0.1.96): Installing...
  • Installing toml (0.10.2): Installing...
  • Installing torch (1.9.0): Installing...
  • Installing transformers (4.8.0): Installing...
  • Installing pytest (5.4.3): Installing...
  • Installing python-dateutil (2.8.1): Installing...
  • Installing pytz (2021.1): Installing...
  • Installing sentencepiece (0.1.96): Installing...
  • Installing toml (0.10.2): Installing...
  • Installing torch (1.9.0): Installing...
  • Installing transformers (4.8.0): Installing...
  • Installing coverage (5.5): Installing...
  • Installing pytest (5.4.3): Installing...
  • Installing python-dateutil (2.8.1): Installing...
  • Installing pytz (2021.1): Installing...
  • Installing sentencepiece (0.1.96): Installing

  • Installing transformers (4.8.0)
  • Installing transformers (4.8.0)
  • Installing torch (1.9.0)
  • Installing transformers (4.8.0)
  • Installing detoxify (0.2.2): Pending...
  • Installing pandas (1.2.5): Pending...
  • Installing pytest-cov (2.12.1): Pending...
  • Installing pandas (1.2.5): Pending...
  • Installing pytest-cov (2.12.1): Pending...
  • Installing detoxify (0.2.2): Installing...
  • Installing pandas (1.2.5): Pending...
  • Installing pytest-cov (2.12.1): Pending...
  • Installing pytest-cov (2.12.1): Installing...
  • Installing pytest-cov (2.12.1): Installing...
  • Installing pandas (1.2.5): Installing...
  • Installing pytest-cov (2.12.1): Installing...
  • Installing pandas (1.2.5): Installing...
  • Installing pytest-cov (2.12.1): Installing...
  • Installing detoxify (0.2.2)
  • Installing pandas (1.2.5): Installing...
  • Installing pytest-cov (2.12.1): Installing...
  • Installing pytest-cov (2.12.1)
  • Installing pytest-cov (2.12.1)
  • Installing pan

Nox recreates the virtual environments from scratch on each invocation (a sensible default). You can speed things up by passing the — reuse-existing-virtualenvs (-r) option:


In [ ]:
# !nox -r 

## 4.5 [pytest-mock](https://github.com/pytest-dev/pytest-mock/)

This plugin provides a mocker fixture which is a thin-wrapper around the patching API provided by the mock package:



In [39]:
#install 
!poetry add --dev pytest-mock -q

Create a detoxify mocker,  

In [40]:
%%writefile tests/test_insult_ai.py

import click.testing
import pytest
from insult_ai import console

@pytest.fixture
def runner():
    """Reusable helper function."""
    return click.testing.CliRunner()

@pytest.fixture
def mock_detoxify_predict(mocker):
    """Generates a mock Detoxify object."""
    mock = mocker.patch("detoxify.Detoxify")
    mock.return_value.predict.return_value = {
        'toxicity': 0., 
        'severe_toxicity': 0., 
        'obscene': 0., 
        'threat': 0., 
        'insult': 0., 
        'identity_hate': 0.
    }
    return mock


def test_insult_me_succeeds(runner, mock_detoxify_predict ):
    result = runner.invoke(console.insult_me, ['--message="You look nice."'])
    assert result.exit_code == 0


Overwriting tests/test_insult_ai.py


In [ ]:
!nox -r 

## 4.6  Marking Tests 

Some tests are too slow to run. So you might want to skip them. 

You can use "pytest.mark" and skip them. 

In [ ]:
%%writefile tests/test_insult_ai.py

import click.testing
import pytest
from insult_ai import console

@pytest.fixture
def runner():
    """Reusable helper function."""
    return click.testing.CliRunner()

@pytest.mark.e2e
def test_insult_me_succeeds(runner):
    result = runner.invoke(console.insult_me, ['--message="You look nice."'])
    assert result.exit_code == 0


Then identify them in the test config file, 

In [ ]:
%%writefile tests/conftest.py

def pytest_configure(config):
    config.addinivalue_line("markers", "e2e: mark as end-to-end test.")

In [ ]:
!poetry run pytest -m "not e2e"

In [ ]:
%%writefile noxfile.py

import nox

@nox.session(python=["3.8"])
def tests(session):
    args = session.posargs or ["--cov", "-m", "not e2e"]
    session.run("poetry", "install", external=True)
    session.run("pytest", *args)

Then you can use parameters to skip the tests, 

In [ ]:
!nox -rs tests-3.8 -- -m e2e

<br><br><br>

# 5. Linting
Linters analyze source code to flag programming errors, bugs, stylistic errors, and suspicious constructs. 


<br><br><br><br><br><br><br><br>
<img src="https://www.pylint.org/pylint.svg" style="height:100px ; float: left;"> 
<br><br>


## 5.1 Pylint 


In [41]:
!pip install pylint -q

You should consider upgrading via the '/Users/rjbrooker/.pyenv/versions/3.9.5/bin/python3.9 -m pip install --upgrade pip' command.


In [45]:
!pylint insult_ai

************* Module insult_ai
insult_ai/__init__.py:1:0: C0114: Missing module docstring (missing-module-docstring)
************* Module insult_ai.console
insult_ai/console.py:2:11: C0303: Trailing whitespace (trailing-whitespace)
insult_ai/console.py:5:19: C0303: Trailing whitespace (trailing-whitespace)
insult_ai/console.py:12:0: C0303: Trailing whitespace (trailing-whitespace)
insult_ai/console.py:15:0: C0303: Trailing whitespace (trailing-whitespace)
insult_ai/console.py:18:0: C0303: Trailing whitespace (trailing-whitespace)
insult_ai/console.py:1:0: C0114: Missing module docstring (missing-module-docstring)
insult_ai/console.py:2:0: W0611: Unused import json (unused-import)
insult_ai/console.py:2:0: C0411: standard import "import json" should be placed before "import click" (wrong-import-order)
insult_ai/console.py:4:0: C0411: third party import "from detoxify import Detoxify" should be placed before "from . import __version__" (wrong-import-order)
insult_ai/console.py:5:0: C0411

Run it on your code, 

In [ ]:
%%writefile -a pyproject.toml

[tool.pylint.messages_control]

max-line-length = 88

disable = [
  "missing-docstring",
  "unused-argument",
  "no-value-for-parameter",
  "no-member",
  "no-else-return",
  "bad-whitespace",
  "bad-continuation",
  "line-too-long",
  "fixme",
  "protected-access",
  "too-few-public-methods",
]

[tool.pylint.design]
# limiting the number of returns might discourage
# the use of guard clauses. So we increase the
# allowed number of returns from 6 to 8
max-returns = 8

## 5.2 Linting + Nox

In [48]:
%%writefile -a noxfile.py

locations = "insult_ai", "tests", "noxfile.py"
@nox.session(python=["3.8", "3.9"])
def lint(session):
    args = session.posargs or locations
    
    # flake 8 
    session.install("pylint", "flake8-black", "flake8-import-order")
    session.run("flake8", *args)
    session.run("pylint", *args)


Appending to noxfile.py


You can create a congif file in order to enabling all the built-in violation classes and setting the complexity limit:


In [ ]:
%%writefile .flake8
[flake8]
select = C,E,F,W
max-complexity = 10
application-import-names = hypermodern_python,tests
import-order-style = google 

You can create pylint config too, 

https://www.codeac.io/documentation/pylint-configuration.html

If we want to run just linting we can, 

In [ ]:
!nox -rs lint


<br><br><br><br><br><br><br><br>
<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTnn4UX2-MpGjn1ip7LZ6skmeSgoedDwVNGCAo2vUNbiBAeOMP4n2BsHNLtQwtyPhYqEes&usqp=CAU" style="height:120px ; float: left;"> 
<br><br><br>

## 5.3 [Black](https://github.com/psf/black)

Black, the uncompromising Python code formatter. 

In [49]:
%%writefile -a noxfile.py

# noxfile.py
@nox.session(python="3.8")
def black(session):
    args = session.posargs or locations
    session.install("black")
    session.run("black", *args)

Appending to noxfile.py


In [50]:
!nox -rs black

nox > Running session black
nox > Re-using existing virtual environment at .nox/black.
nox > python -m pip install black
nox > black insult_ai tests noxfile.py
reformatted insult_ai/__init__.py
reformatted insult_ai/console.py
reformatted tests/test_insult_ai.py
reformatted noxfile.py
All done! ✨ 🍰 ✨
4 files reformatted, 1 file left unchanged.
nox > Session black was successful.


In [ ]:
!nox -rs lint

# 5.4 [Bandit](https://pypi.org/project/bandit/)

Identifying security issues. 


In [ ]:
%%writefile -a noxfile.py

@nox.session(python=["3.8",])
def bandit(session):
    args = session.posargs or locations
    session.install(
        "flake8",
        "flake8-bandit",
    )
    session.run("flake8", *args)


In [ ]:
%%writefile .flake8
[flake8]
select = B,B9,BLK,C,E,F,I,W,S
max-complexity = 10
application-import-names = hypermodern_python,tests
import-order-style = google 

In [ ]:
!nox -rs black
!nox -rs bandit



<br><br><br><br><br><br><br><br>
<img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/25b60c01ce72664af24ec46d17813024c247384a/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f70797570696f2f7361666574792f6d61737465722f7361666574792e6a7067
" style="height:120px ; float: left;"> 
<br><br><br>

### 5.5 Safty




In [51]:
!pip install safety -q


You should consider upgrading via the '/Users/rjbrooker/.pyenv/versions/3.9.5/bin/python3.9 -m pip install --upgrade pip' command.


In [ ]:
!pip uninstall insecure-package -y

Cehck security, 

In [52]:
!safety check

+==============================================================================+
|                                                                              |
|                               /$$$$$$            /$$                         |
|                              /$$__  $$          | $$                         |
|           /$$$$$$$  /$$$$$$ | $$  \__//$$$$$$  /$$$$$$   /$$   /$$           |
|          /$$_____/ |____  $$| $$$$   /$$__  $$|_  $$_/  | $$  | $$           |
|         |  $$$$$$   /$$$$$$$| $$_/  | $$$$$$$$  | $$    | $$  | $$           |
|          \____  $$ /$$__  $$| $$    | $$_____/  | $$ /$$| $$  | $$           |
|          /$$$$$$$/|  $$$$$$$| $$    |  $$$$$$$  |  $$$$/|  $$$$$$$           |
|         |_______/  \_______/|__/     \_______/   \___/   \____  $$           |
|                                                          /$$  | $$           |
|                                                         |  $$$$$$/           |
|  by pyup.io               

In [ ]:
!pip uninstall insecure-package -y


<br><br><br><br><br><br><br><br>
<img src="https://camo.githubusercontent.com/c0bc16116647eb3c773360c495d8537d509df514fa8f77b545fca2edde5fc3d7/68747470733a2f2f6861646f6c696e742e6769746875622e696f2f6861646f6c696e742f696d672f6361745f636f6e7461696e65722e706e67
" style="height:120px ; float: left;"> 
<br><br><br>

## 5.4 Docker Linting

In [ ]:
!brew install hadolint

In [53]:
!mkdir docker

In [54]:
%%writefile docker/Dockerfile

FROM debian
RUN export node_version="0.10" \
&& apt-get update && apt-get -y install nodejs="$node_verion"
COPY package.json usr/src/app
RUN cd /usr/src/app \
&& npm install node-static

Writing docker/Dockerfile


In [55]:
!hadolint docker/Dockerfile

docker/Dockerfile:2 DL3006 warning: Always tag the version of an image explicitly
docker/Dockerfile:3 SC2154 warning: node_verion is referenced but not assigned (did you mean 'node_version'?).
docker/Dockerfile:3 DL3009 info: Delete the apt-get lists after installing something
docker/Dockerfile:3 DL3015 info: Avoid additional packages by specifying `--no-install-recommends`
docker/Dockerfile:5 DL3045 warning: `COPY` to a relative destination without `WORKDIR` set.
docker/Dockerfile:6 DL3003 warning: Use WORKDIR to switch to a directory
docker/Dockerfile:6 DL3016 warning: Pin versions in npm. Instead of `npm install <package>` use `npm install <package>@<version>`


In [56]:
!rm docker/Dockerfile


<br><br><br><br><br><br><br><br>
<img src="https://raw.githubusercontent.com/prettier/prettier-logo/master/images/prettier-banner-light.png
" style="height:150px ; float: left;"> 
<br><br><br>

## 5.4 prettier - HTML, Markdown, Yaml (JS, CSS...)


In [ ]:
!brew install prettier

In [ ]:
!prettier --check .

In [ ]:
!ls


<br><br><br><br><br><br><br><br>
<img src="https://api.coala.io/en/latest/_static/images/coala_logo.svg
" style="height:150px ; float: left;"> 
<br><br><br>

## 5.4 Coala

In [57]:
!poetry run pip install coala-bears 

  Using cached PyYAML-3.13-cp39-cp39-macosx_11_0_x86_64.whl
  Using cached click-6.6-py2.py3-none-any.whl (71 kB)
  Using cached colorama-0.3.9-py2.py3-none-any.whl (20 kB)


  Using cached Sphinx-1.4.9-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: sphinx
    Found existing installation: Sphinx 4.0.2
    Uninstalling Sphinx-4.0.2:
      Successfully uninstalled Sphinx-4.0.2
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstalled PyYAML-5.4.1
  Attempting uninstall: click
    Found existing installation: click 8.0.1
    Uninstalling click-8.0.1:
      Successfully uninstalled click-8.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx-autodoc-typehints 1.12.0 requires Sphinx>=3.0, but you have sphinx 1.4.9 which is incompatible.
insult-ai 0.1.1 requires click<9.0.0,>=8.0.1, but you have click 6.6 which is incompatible.


You should consider upgrading via the '/Users/rjbrooker/Library/Caches/pypoetry/virtualenvs/insult-ai-nTe-_8iu-py3.9/bin/python -m pip install --upgrade pip' command.


In [60]:
%%writefile  .coafile
[all]
enabled = True
overridable = 3
files = insult_ai/**.py
bears=PEP8Bear,PyUnusedCodeBear,PyLintBear

Overwriting .coafile


In [63]:
!poetry run coala 

Executing section all...
Executing section cli...



<br><br><br><br><br><br><br><br>
<img src="https://pre-commit.com/logo.svg
" style="height:150px ; float: left;"> 
<br><br><br>

## 5.4 [pre-commit](https://pre-commit.com/)



In [ ]:
!pip install pre-commit

In [ ]:
%%writefile  .pre-commit-config.yaml

repos:
-   repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v2.3.0
    hooks:
    -   id: check-yaml
    -   id: end-of-file-fixer
    -   id: trailing-whitespace
-   repo: https://github.com/psf/black
    rev: 19.3b0
    hooks:
    -   id: black

In [ ]:
!pre-commit install

now pre-commit will run automatically on git commit


## 5.4 [Pycodestyle](https://pycodestyle.pycqa.org/en/latest/)

Linting in Jupyter notebooks.

In [ ]:
!pip install pycodestyle pycodestyle_magic flake8 -q

In [66]:
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [67]:
%%pycodestyle

def square_of_number(
     num1, num2, num3, 
     num4):
    return num1**2, num2**2, num3*


4:23: W291 trailing whitespace
7:1: W391 blank line at end of file


# 6. Typing 

## 6.1 Type Hints

In [ ]:
from typing import List

# This is a variable holding an integer.
a : int = 42

# This is a function which accepts and returns an integer.
def increment(number: List[int]) -> int:
    return sum(number)

## 6.2 Type Annotations

In [ ]:
from __future__ import annotations
from typing import Annotated
from typing import List

# create a new type
Intlist = Annotated[ List[int], 'Some meaningless list of integers.' ]

def sum_list( numbers: Intlist ) -> int:
    return sum(numbers)


<br><br><br><br><br><br><br><br>
<img src="https://camo.githubusercontent.com/20e0f72b4f84dc5b42aceb95eb8eaa6c574746c0057e9e2525dd6cb4797d565f/687474703a2f2f6d7970792d6c616e672e6f72672f7374617469632f6d7970795f6c696768742e737667" style="height:50px ; float: left;"> 

## 6.3 [Mypy](https://github.com/python/mypy) - : Optional Static Typing for Python

Static type checking with mypy


In [ ]:
# install 
!poetry add --dev mypy -q

Call mypy from nox,

In [68]:
%%writefile -a noxfile.py

@nox.session(python=["3.8", "3.9"])
def mypy(session) -> None:
    args = session.posargs or locations
    session.install("mypy")
    session.run("mypy", *args)

Appending to noxfile.py


Mypy raises an error if it cannot find any type definitions for a Python package used by your program. 

In [ ]:
%%writefile mypy.ini
[mypy]
ignore_missing_imports = True

[mypy-nox.*,pytest]
ignore_missing_imports = True

In [ ]:
!nox -rs mypy

## 6.4 [pytype](https://github.com/google/pytype) 🦆✔

Pytype checks and infers types for your Python code - without requiring type annotations.  It infer types on code even when the code has no type hints on it.

You need a Python 3.6-3.8 interpreter to run pytype. 

In [ ]:
!poetry add --dev --python=3.8 pytype -q

In [69]:
%%writefile -a noxfile.py

@nox.session(python="3.8")
def pytype(session):
    """Run the static type checker."""
    args = session.posargs or ["--disable=import-error", *locations]
    session.install("pytype")
    session.run("pytype", *args)

Appending to noxfile.py


In [70]:
%%writefile insult_ai/tmp.py

def f():
    return "2021"

def g():
    return f() + 6


Writing insult_ai/tmp.py


In [71]:
!nox -rs pytype

nox > Running session pytype
nox > Creating virtual environment (virtualenv) using python3.8 in .nox/pytype
nox > python -m pip install pytype
nox > pytype --disable=import-error insult_ai tests noxfile.py
Computing dependencies
Analyzing 6 sources with 0 local dependencies
ninja: Entering directory `.pytype'
[6/6] check insult_ai.tmpt_ai
FAILED: /Users/rjbrooker/Documents/github_repos/modern-python-tutorial/insult-ai/.pytype/pyi/insult_ai/tmp.pyi 
/Users/rjbrooker/Documents/github_repos/modern-python-tutorial/insult-ai/.nox/pytype/bin/python -m pytype.single --disable import-error --imports_info /Users/rjbrooker/Documents/github_repos/modern-python-tutorial/insult-ai/.pytype/imports/insult_ai.tmp.imports --module-name insult_ai.tmp -V 3.8 -o /Users/rjbrooker/Documents/github_repos/modern-python-tutorial/insult-ai/.pytype/pyi/insult_ai/tmp.pyi --analyze-annotated --nofail --quick /Users/rjbrooker/Documents/github_repos/modern-python-tutorial/insult-ai/insult_ai/tmp.py
File "/Users/rjbr

In [72]:
!rm insult_ai/tmp.py
!nox -rs pytype

nox > Running session pytype
nox > Re-using existing virtual environment at .nox/pytype.
nox > python -m pip install pytype
nox > pytype --disable=import-error insult_ai tests noxfile.py
Computing dependencies
Analyzing 5 sources with 0 local dependencies
ninja: Entering directory `.pytype'
ninja: no work to do.
Leaving directory '.pytype'
Success: no errors found
nox > Session pytype was successful.


## 6.7 [Pydantic](https://pydantic-docs.helpmanual.io/)

In [ ]:
!pip install pydantic -q

Simple type validations, 

In [73]:
from pydantic import validate_arguments

@validate_arguments
def pos_or_kw(a: int, b: int = 2) -> str:
    return f'a={a} b={b}'

In [74]:
pos_or_kw('a','5')

ValidationError: 1 validation error for PosOrKw
a
  value is not a valid integer (type=type_error.integer)

More complicated validations, 

In [75]:
from pydantic import validate_arguments
from pydantic import BaseModel


class User(BaseModel):
    a: int
    b: int = 1

@validate_arguments
def pos_or_kw( values : User ) -> str:
    return f'a={a} b={b}'.format(**values.dict())

In [76]:
pos_or_kw({'a':1, 'b': 'a'})

ValidationError: 1 validation error for PosOrKw
values -> b
  value is not a valid integer (type=type_error.integer)

## 6.8 [Typeguard](https://github.com/agronholm/typeguard) - Runtime type checking

It checks that arguments match parameter types of annotated functions as your program is being executed


In [ ]:
!poetry add --dev typeguard

In [ ]:
!tree

In [ ]:
%%writefile tests/test_insult_ai.py
import click.testing
import pytest
from insult_ai import console

def test_insult_me_succeeds():
    console.insult_me(1)


In [ ]:
%%writefile -a noxfile.py

package = "insult_ai"
@nox.session(python=["3.8", "3.9"])
def typeguard(session):
    args = session.posargs or ["-m", "not e2e"]
    session.run("poetry", "install", "--no-dev", external=True)
    session.install("pytest", "pytest-mock", "typeguard")
    session.run("pytest", f"--typeguard-packages={package}", *args)

In [ ]:
!nox -rs typeguard


## 6.9 flake8-annotations

Detects the absence of PEP 3107-style function annotations

In [ ]:
!poetry add --dev flake8-annotations -q

In [ ]:
%%writefile -a noxfile.py

@nox.session(python=["3.8", "3.9"])
def lint(session):
    args = session.posargs or locations
    session.install(
        "flake8",
        "flake8-annotations",
        "flake8-bandit",
        "flake8-black",
        "flake8-bugbear",
        "flake8-import-order",
    )
    session.run("flake8", *args)

In [ ]:
%%writefile .flake8
[flake8]
select = ANN,B,B9,BLK,C,E,F,I,S,W
per-file-ignores =
    tests/*:S101,ANN
    noxfile.py:ANN

In [ ]:
%%writefile .flake8
[MASTER]
init-hook="from pylint.config import find_pylintrc; import os, sys; sys.path.append(os.path.dirname(find_pylintrc()))"

In [ ]:
!nox -rs lint

## 6.10 Python docstrings

Style types,
#### Google  - https://google.github.io/styleguide/pyguide.html

In [ ]:

"""
This is an example of Google style.

Args:
    param1: This is the first param.
    param2: This is a second param.

Returns:
    This is a description of what is returned.

Raises:
    KeyError: Raises an exception.
"""

#### reST

In [ ]:
"""
This is a reST style.

:param param1: this is a first param
:param param2: this is a second param
:returns: this is a description of what is returned
:raises keyError: raises an exception
"""

#### Numpydoc - https://numpydoc.readthedocs.io/en/latest/

In [ ]:
"""
My numpydoc description of a kind
of very exhautive numpydoc format docstring.

Parameters
----------
first : array_like
    the 1st param name `first`
second :
    the 2nd param
third : {'value', 'other'}, optional
    the 3rd param, by default 'value'

Returns
-------
string
    a value in a string

Raises
------
KeyError
    when a key error
OtherError
    when an other error
"""

<br><br><br>
Add some docstrings,

In [ ]:
!open insult_ai/__init__.py

In [102]:
%%writefile insult_ai/__init__.py
"""Insult-ai python package."""
__version__ = '0.2.0'

Overwriting insult_ai/__init__.py


In [78]:
%%writefile insult_ai/console.py

"""Command-line interface. A command line tool for using Detoxify.py"""
import click
from . import __version__
from detoxify import Detoxify
import pandas as pd 

@click.command()
@click.option('--message', default="", help='A message to send to insult-ai.')
@click.version_option(version=__version__)
def insult_me(
        message : str 
    ):
    """
    A command line tool for .
    
    Calls Detoxify.py 'original' model and returns the results. 
    
    Args:
        message: A message to be sent to the sentiment analysis model.
    
    Returns:
        None
    
    """
    
    #load model
    model = Detoxify('original')
    
    #predict toxicity
    results = model.predict(message)
    
    #echo results
    click.echo(pd.Series(results))


Overwriting insult_ai/console.py


## 6.11 Flake8-docstrings

Check that docstrings are compliant with the style recommendations of PEP 257. Warnings range from missing docstrings to issues with whitespace, quoting, and docstring content.


In [ ]:
!poetry add --dev flake8-docstrings -q

In [ ]:
%%writefile noxfile.py

import nox
from nox.sessions import Session

locations = "insult_ai", "tests", "noxfile.py"

@nox.session(python=["3.8", "3.9"])
def lint(session) -> None:
    args = session.posargs or locations
    session.install('black')
    session.run("black", *args)
    session.install(
        "flake8",
        "flake8-annotations",
        "flake8-bandit",
        "flake8-black",
        "flake8-bugbear",
        "flake8-docstrings",
        "flake8-import-order",
    )
    session.run("flake8", *args)

In [ ]:
%%writefile .flake8
select = ANN,B,B9,BLK,C,D,E,F,I,S,W
docstring-convention = google

In [ ]:
!nox -rs lint

<br><br><br><br><br><br><br><br>
<img src="https://camo.githubusercontent.com/b38b75eb87e0170e4a72323babb2ec8c00a467c6509d862b39b7497af9d3f57a/68747470733a2f2f692e696d6775722e636f6d2f7530745959784d2e706e67" style="height:100px ; float: left;"> 
# 6.12. xdoctest

In [123]:
%%writefile insult_ai/console.py
"""Command-line interface. A command line tool for using Detoxify.py"""
import click
from . import __version__
from detoxify import Detoxify
import pandas as pd 

@click.command()
@click.version_option(version=__version__)
@click.option('--message', default="", help='A message to send to insult-ai.')
@click.version_option(version=__version__)
def insult_me(
        message : str 
    ):
    """
    A command line tool for .
    
    Calls Detoxify.py 'original' model and returns the results. 
    
    Args:
        message: A message to be sent to the sentiment analysis model.
    
    Returns:
        None
    
    Example:
        >>> import click.testing
        >>> runner = click.testing.CliRunner()
        >>> result = runner.invoke(insult_me, ['--message="You look nice."'])
        >>> print(result.exit_code)
        0
    """
    
    #load model
    model = Detoxify('original')
    
    #predict toxicity
    results = model.predict(message)
    
    #echo results
    click.echo(pd.Series(results))


Overwriting insult_ai/console.py


In [104]:
!poetry add --dev xdoctest -q

In [105]:
%%writefile -a noxfile.py

@nox.session(python=["3.9"])
def xdoctest(session) -> None:
    """Run examples with xdoctest."""
    args = session.posargs or ["all"]
    session.run("poetry", "install", "--no-dev", external=True)
    session.install("xdoctest", "pygments")
    session.run("python", "-m", "xdoctest", 'insult_ai', *args)


Appending to noxfile.py


In [ ]:
!nox -rs xdoctest 

<br><br><br><br><br><br><br><br>
<img src="https://i0.wp.com/blog.fossasia.org/wp-content/uploads/2017/07/sphinxdoc-450.png?fit=800%2C800&ssl=1" style="height:150px ; float: left;"> 
<br><br><br>

## 6.13 [sphinx](https://www.sphinx-doc.org/en/master/)

In [82]:
!poetry add --dev sphinx

Using version ^4.0.2 for Sphinx

Updating dependencies
Resolving dependencies... (7.0s)Resolving dependencies... (4.8s)Resolving dependencies... (6.9s)Resolving dependencies... (7.2s)

Writing lock file

Package operations: 0 installs, 1 update, 0 removals

  • Updating sphinx (1.4.9 -> 4.0.2): Pending...
  • Updating sphinx (1.4.9 -> 4.0.2): Installing...
  • Updating sphinx (1.4.9 -> 4.0.2)


Create the Sphinx configuration file docs/conf.py. This provides meta information about your project:


In [83]:
!mkdir docs

In [84]:
%%writefile docs/index.rst

This is docs/index.rst,
documenting the Hypermodern Python project.

Writing docs/index.rst


In [85]:
%%writefile docs/conf.rst

"""Sphinx configuration."""

project = "insult-ai"
author = "Richard Brooker"
copyright = f"2020, {author}" 

Writing docs/conf.rst


Add a Nox session to build the documentation:


In [86]:
%%writefile -a noxfile.py

locations = "insult_ai", "tests", "noxfile.py", "docs/conf.py"

@nox.session(python=["3.9"])
def docs(session) -> None:
    """Build the documentation."""
    session.install("sphinx", "sphinx_autodoc_typehints")
    session.run("sphinx-build", "docs", "docs/_build")

Appending to noxfile.py


## 6.1 sphinx-autodoc-typehints
Sphinx to generate API documentation from the documentation strings and type annotations in the package, using three Sphinx extensions:

- [autodoc](https://github.com/heavenshell/py-autodoc) enables Sphinx to generate API documentation from the docstrings in your package.
- [napoleon](https://sphinxcontrib-napoleon.readthedocs.io/en/latest/) pre-processes Google-style docstrings to reStructuredText.
- [sphinx-autodoc-typehints](https://github.com/agronholm/sphinx-autodoc-typehints) uses type annotations to document the types of function parameters and return values.


In [110]:
# the only package that needs installing 
!poetry add --dev sphinx-autodoc-typehints sphinx-click -q

  • Installing sphinx-click (3.0.1): Pending...
  • Installing sphinx-click (3.0.1): Downloading... 0%
  • Installing sphinx-click (3.0.1): Downloading... 100%
  • Installing sphinx-click (3.0.1): Downloading... 100%
  • Installing sphinx-click (3.0.1): Installing...
  • Installing sphinx-click (3.0.1)


Add it to your nox file,

In [88]:
%%writefile -a noxfile.py

@nox.session(python="3.9")
def docs(session) -> None:
    """Build the documentation."""
    session.run("poetry", "install", "--no-dev", external=True)
    session.install( "sphinx", "sphinx-autodoc-typehints",'sphinx-click')
    session.run("sphinx-build", "docs", "docs/_build")

Appending to noxfile.py


In [111]:
%%writefile docs/conf.py
import sys
import os 

sys.path.insert(0, os.path.abspath('../../'))

extensions = [
    "sphinx.ext.autodoc",
    "sphinx.ext.napoleon",
    "sphinx_autodoc_typehints",
    'sphinx_click'
]

Overwriting docs/conf.py


Create the file docs/reference.rst, containing the API reference for the project:

You can now reference docstrings in your Sphinx documentation using directives such as automodule, autoclass, and autofunction.


In [121]:
%%writefile docs/reference.rst
Reference
=========

.. contents::
        :local:
        :backlinks: none


insult_ai.console
--------------------------

.. click:: insult_ai.console:insult_me
  :prog: insult-ai


Overwriting docs/reference.rst


- The automodule directive inserts the documentation for the specified Python module. With the :members: option, it also includes documentation for the classes and functions defined by the module.

- The contents directive inserts a table of content into the document. The :local: option avoids including the page title in the table of contents. The :backlinks: none option avoids linking each section title to the table of contents.



Include the new file in the navigation sidebar,

In [119]:
%%writefile docs/index.rst

Insult-AI Python Project
==============================

.. toctree::
    :hidden:
    :maxdepth: 2
    
    reference

Overwriting docs/index.rst


In [124]:
!nox -rs docs

nox > Running session docs
nox > Re-using existing virtual environment at .nox/docs.
nox > poetry install --no-dev
Installing dependencies from lock file

Package operations: 0 installs, 0 updates, 17 removals

  • Removing alabaster (0.7.12): Pending...
  • Removing alabaster (0.7.12): Removing...
  • Removing alabaster (0.7.12)
  • Removing babel (2.9.1): Pending...
  • Removing babel (2.9.1): Removing...
  • Removing babel (2.9.1)
  • Removing docutils (0.17.1): Pending...
  • Removing docutils (0.17.1): Removing...
  • Removing docutils (0.17.1)
  • Removing imagesize (1.2.0): Pending...
  • Removing imagesize (1.2.0): Removing...
  • Removing imagesize (1.2.0)
  • Removing jinja2 (3.0.1): Pending...
  • Removing jinja2 (3.0.1): Removing...
  • Removing jinja2 (3.0.1)
  • Removing markupsafe (2.0.1): Pending...
  • Removing markupsafe (2.0.1): Removing...
  • Removing markupsafe (2.0.1)
  • Removing pygments (2.9.0): Pending...
  • Removing pygments (2.9.0): Removing...
  • Removin

The Sphinx documentation contains a good introduction to

https://www.sphinx-doc.org/en/master/usage/restructuredtext/index.html
    

### 6.1.2  [ReadTheDocs.ord](https://readthedocs.org/)
Read the Docs simplifies software documentation by automating building, versioning, and hosting of your docs for you.



<br><br><br><br><br><br><br><br><br><br><br><br>

In [1]:
from IPython.core.display import HTML
HTML("""
<style>
h1 { margin-top:190px !important ; border-top: 1px solid rgba(0,0,0,0.01)}
h2,h3 { margin-top:150px !important }
h4 {margin-top:80px !important}
</style>
""")